In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
import holidays
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt
os.getcwd()

'/Users/admin/Desktop/DS4BThesisSNCB'

In [2]:
stops_post_covid = pd.read_parquet("~/Downloads/all_stops_ungrouped/all_stops_post_covid.parquet")
df = stops_post_covid
PTCAR = pd.read_csv("/Users/admin/Desktop/DS4BThesisSNCB/data/PTCAR.csv",sep=";")
df = df.merge(PTCAR,on='REF_PTCAR_NO',how = 'left')
df

,REF_TOD_TRAIN_ID,TRAIN_NUMBER,TRAIN_DEPARTURE_DATE,REF_PTCAR_NO,PTCAR_ORD_NO,THR_ARR,THR_DEP,HR_ARR,HR_DEP,EH_ARR,...,p_indicator,SYMBOLIC_NAME,GPS_LATITUDE,GPS_LONGITUDE,LONG_NAME_FR,LONG_NAME_NL,SHORT_NAME_FR,SHORT_NAME_NL,UIC_CODE,CLASSIFICATION
0,30429744,13251,2022-06-05,1174,106,2022-06-05 13:52:00,2022-06-05 13:52:00,2022-06-05 13:51:57,2022-06-05 13:51:57,-2,...,0,FEM,50.900519,4.633522,VELTEM,VELTEM,VELTEM,VELTEM,11270.0,4.0
1,30429744,13251,2022-06-05,368,107,2022-06-05 13:55:00,2022-06-05 13:56:00,2022-06-05 13:54:15,2022-06-05 13:56:02,-44,...,0,MRP,50.896506,4.584997,ERPS-KWERPS,ERPS-KWERPS,ERPS-KWERPS,ERPS-KWERPS,11262.0,4.0
2,30429744,13251,2022-06-05,648,109,2022-06-05 13:59:00,2022-06-05 14:00:00,2022-06-05 13:58:45,2022-06-05 14:00:11,-14,...,0,FTB,50.893072,4.543297,KORTENBERG,KORTENBERG,KORTENBERG,KORTENBERG,11254.0,4.0
3,30429744,13251,2022-06-05,1260,114,2022-06-05 14:06:00,2022-06-05 14:07:00,2022-06-05 14:05:32,2022-06-05 14:06:55,-27,...,0,FZA,50.885720,4.469886,ZAVENTEM,ZAVENTEM,ZAVENTEM,ZAVENTEM,11221.0,4.0
4,30429744,13251,2022-06-05,325,115,2022-06-05 14:09:00,2022-06-05 14:10:00,2022-06-05 14:08:50,2022-06-05 14:10:18,-9,...,0,FDG,50.890481,4.441931,DIEGEM,DIEGEM,DIEGEM,DIEGEM,11213.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12081260,31636558,4338,2022-12-10,840,106,2022-12-10 17:49:00,2022-12-10 17:58:00,2022-12-10 17:57:44,2022-12-10 18:03:24,524,...,0,LML,51.190668,5.115016,MOL,MOL,MOL,MOL,32409.0,1.0
12081261,31329807,4034,2022-10-28,218,116,2022-10-28 13:05:00,2022-10-28 13:10:00,2022-10-28 13:05:14,2022-10-28 13:09:55,14,...,0,LX,50.838860,4.373820,BRUXELLES-LUXEMBOURG,BRUSSEL-LUXEMBURG,BRUX-LUXEMB,BRUSS-LUXEMB,11304.0,1.0
12081262,31329661,2627,2022-10-28,715,117,2022-10-28 06:13:00,2022-10-28 06:24:00,2022-10-28 06:13:49,2022-10-28 06:24:16,49,...,0,FLV,50.881356,4.716096,LEUVEN,LEUVEN,LEUVEN,LEUVEN,33001.0,1.0
12081263,31326770,5468,2022-10-28,726,106,2022-10-28 19:02:00,2022-10-28 19:06:00,2022-10-28 19:28:05,2022-10-28 19:29:49,1565,...,0,FL,50.625482,5.565207,LIEGE-GUILLEMINS,LIEGE-GUILLEMINS,LIEGE-GUILL,LIEGE-GUILL,41004.0,1.0


In [3]:
# one-hot-encode LONG_NAME_NL column
one_hot_encoded = pd.get_dummies(df['LONG_NAME_NL'])
one_hot_encoded
# concatenate one-hot-encoded columns with original dataframe
df_ohe = pd.concat([df, one_hot_encoded], axis=1)
# group by REF_TOD_TRAIN_ID and get the maximum value of each station
grouped_df = df_ohe.groupby('TRAIN_NUMBER')[one_hot_encoded.columns.tolist()].max()
grouped_stops = grouped_df.reset_index()
grouped_stops

,AALST,AALST-KERREBROEK,AALTER,AARSCHOT,AARSELE,ACREN,AISEAU,ALKEN,AMAY,AMPSIN,...,ZELE,ZELLIK,ZICHEM,ZINGEM,ZOLDER,ZOLDER-KRUISSPOREN,ZONHOVEN,ZONHOVEN-ROOSTER P/U,ZOTTEGEM,ZWIJNDRECHT
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12081260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12081261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12081262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12081263,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
grouped_stops = grouped_stops.reset_index()

In [115]:
train_station = grouped_stops
#train_station.set_index('TRAIN_NUMBER', inplace=True)

In [116]:
train_station = train_station.drop(columns= ['level_0','index'])
train_station[['TRAIN_NUMBER','AALST','AALTER']]


,TRAIN_NUMBER,AALST,AALTER
0,11,0,0
1,13,0,0
2,15,0,0
3,17,0,0
4,19,0,0
...,...,...,...
5642,19968,0,0
5643,19970,0,0
5644,19972,0,0
5645,19974,0,0


In [165]:
# Extract the train numbers and the station columns from the dataframe
train_numbers = train_station['TRAIN_NUMBER']
stations = train_station.columns[1:]

# Compute a binary matrix indicating which trains pass through each station
station_matrix = train_station[stations].values.astype(bool)

# Compute the adjacency matrix using a dot product
adj_matrix = pd.DataFrame(station_matrix.dot(station_matrix.T), index=train_numbers, columns=train_numbers)

# Fill the diagonal with zeros
np.fill_diagonal(adj_matrix.values, 0)

# Print the adjacency matrix

adj_matrix = adj_matrix.astype(int)
#adj_matrix.columns = adj_matrix.columns.map(str)
adj_matrix = adj_matrix.reset_index()
adj_matrix.set_index('TRAIN_NUMBER', inplace=True)
adj_matrix

TRAIN_NUMBER,11,13,15,17,19,315,317,319,404,405,...,19958,19960,19962,19964,19966,19968,19970,19972,19974,19976
TRAIN_NUMBER,,,,,,,,,,,,,,,,,,,,,
11,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
13,1,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
15,1,1,0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
17,1,1,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
19,1,1,1,1,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19968,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,1,1,1,1
19970,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,1,1,1
19972,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,1,1


In [166]:
# calculate the node degree, betweenness, and degree centrality
graph = nx.Graph(adj_matrix)
degree = dict(graph.degree())
betweenness = nx.betweenness_centrality(graph)
centrality = nx.degree_centrality(graph)
pagerank =  nx.pagerank(graph) # use pagerank_numpy instead of pagerank


# Combine the results into a dataframe
network_char = pd.DataFrame({'TRAIN_NUMBER': list(graph.nodes()),
                             'Degree': list(degree.values()),
                             'Betweenness': list(betweenness.values()),
                             'Centrality': list(centrality.values()),
                             'Pagerank': list(pagerank.values())})

network_char

KeyboardInterrupt: 

In [ ]:
print(network_char[network_char['Degree'] == network_char['Degree'].max()][['LONG_NAME_NL', 'Degree']])
print(network_char[network_char['Betweenness'] == network_char['Betweenness'].max()][['LONG_NAME_NL', 'Betweenness']])
print(network_char[network_char['Centrality'] == network_char['Centrality'].max()][['LONG_NAME_NL', 'Centrality']])
print(network_char[network_char['Pagerank'] == network_char['Pagerank'].max()][['LONG_NAME_NL', 'Pagerank']])

In [ ]:
network_char.sort_values(by="Degree", ascending=False)

In [ ]:
network_char.sort_values(by="Betweenness", ascending=False)

In [ ]:
network_char.sort_values(by="Centrality", ascending=False)

In [ ]:
network_char.sort_values(by="Pagerank", ascending=False)

In [ ]:
network_char.to_csv("/Users/admin/Desktop/DS4BThesisSNCB/data/journey_network.csv", index=False)